# Linguaggio pixel

Si consideri il nucleo di un semplice linguaggio di programmazione funzionale, la cui sintassi è descritta da

_Pixel_ $p ::= \langle r, g, b \rangle \text{ dove } r, g, b \in \{0, 1, . . . , 255\}$

_Identificatori_ $I ::= \dots$

_Espressioni_ $e ::= I | p | \text{lighten } e | \text{darken } e | \text{let } I = e_1 \text{ in } e_2$

Intuitivamente, un pixel è un tipo di dato che contiene tre valori interi compresi tra 0 e 255 (il primo valore codifica red, il secondo green e il terzo blue).
L'espressione `lighten` produce come risultato un pixel dove ogni componente del pixel passato come argomento viene incrementato di 1. L'incremento del valore 255 produce 255. L'espressione `darken` produce come risultato un pixel dove ogni componente del pixel passato come argomento viene diminuito di 1. Il decremento del valore 0 produce 0.

Si definisca l’interprete del linguaggio utilizzando OCaml come linguaggio di implementazione.

### Tipi e sintassi

In [1]:
type pixel = int * int * int;;

let getr (r, _, _) = r;;
let getg (_, g, _) = g;;
let getb (_, _, b) = b;;

let checkPixel ((r, g, b): pixel) = 0 <= r && r < 256 && 0 <= g && g < 256 && 0 <= b && b < 256;;

type pixel = int * int * int


val getr : 'a * 'b * 'c -> 'a = <fun>


val getg : 'a * 'b * 'c -> 'b = <fun>


val getb : 'a * 'b * 'c -> 'c = <fun>


val checkPixel : pixel -> bool = <fun>


In [2]:
type tname = TPixel | TUnbound;;

type ide = string;;

type expr =
  | Ide of ide
  | P of pixel
  | Lighten of expr
  | Darken of expr
  | Let of ide * expr * expr;;

type tname = TPixel | TUnbound


type ide = string


type expr =
    Ide of ide
  | P of pixel
  | Lighten of expr
  | Darken of expr
  | Let of ide * expr * expr


### Valori esprimibili e type checking

In [3]:
type evT =
  | Pixel of pixel
  | Unbound;;

let typecheck (t: tname) (v: evT): bool =
  match t, v with
    | TPixel, Pixel(p) -> checkPixel p
    | TUnbound, Unbound -> true
    | _, _ -> false;;

type evT = Pixel of pixel | Unbound


val typecheck : tname -> evT -> bool = <fun>


### Interprete

In [4]:
type env = ide -> evT;;

let emptyenv: env = fun x -> Unbound;;

(* Estende l'ambiente s, aggiungendo il legame dell'identificatore x al valore v,
 * eventualmente nascondendo un legame precedente.
 *)
let bind (s: env) (x: ide) (v: evT): env = fun i -> if i = x then v else s i;;

type env = ide -> evT


val emptyenv : env = <fun>


val bind : env -> ide -> evT -> env = <fun>


In [5]:
let incrChannel n = if n = 255 then 255 else n + 1;;

let decrChannel n = if n = 0 then 0 else n - 1;;

exception Err of string

let rec eval (e: expr) (s: env): evT =
  match e with
    | Ide(x) -> s x
    | P(p) -> if checkPixel p then Pixel(p) else raise (Err "errore: non è un pixel")
    | Lighten(e) ->
      let res = eval e s in
        if typecheck TPixel res then
          match res with
            | Pixel(p) -> Pixel (incrChannel (getr p), incrChannel (getg p), incrChannel (getb p))
            | _ -> raise (Err "il typecheck è buggato")
        else
          raise (Err "errore: non è un pixel")
    | Darken(e) ->
      let res = eval e s in
        if typecheck TPixel res then
          match res with
            | Pixel(p) -> Pixel (decrChannel (getr p), decrChannel (getg p), decrChannel (getb p))
            | _ -> raise (Err "il typecheck è buggato")
        else
          raise (Err "errore: non è un pixel")
    | Let(x, e1, e2) -> eval e2 (bind s x (eval e1 s));;

val incrChannel : int -> int = <fun>


val decrChannel : int -> int = <fun>


exception Err of string


val eval : expr -> env -> evT = <fun>


In [6]:
eval (Lighten (P (10, 151, 255))) emptyenv;;

- : evT = Pixel (11, 152, 255)


## Funzioni

Estendiamo il linguaggio con la possibilità di definire funzioni unarie e non ricorsive:

_Espressioni_ $e ::= \dots | \text{fun } I\, e | \text{apply } e_1 e_2$

In [7]:
type tname = TPixel | TClosure | TUnbound;;

type ide = string;;

type expr =
  | Ide of ide
  | P of pixel
  | Lighten of expr
  | Darken of expr
  | Let of ide * expr * expr
  | Fun of ide * expr
  | Apply of expr * expr;;

type tname = TPixel | TClosure | TUnbound


type ide = string


type expr =
    Ide of ide
  | P of pixel
  | Lighten of expr
  | Darken of expr
  | Let of ide * expr * expr
  | Fun of ide * expr
  | Apply of expr * expr


In [8]:
type evT =
  | Pixel of pixel
  | Closure of ide * expr * env
  | Unbound
and
  env = ide -> evT;;

let typecheck (t: tname) (v: evT): bool =
  match t, v with
    | TPixel, Pixel(p) -> checkPixel p
    | TUnbound, Unbound -> true
    | TClosure, Closure(_, _, _) -> true
    | _, _ -> false;;

type evT = Pixel of pixel | Closure of ide * expr * env | Unbound
and env = ide -> evT


val typecheck : tname -> evT -> bool = <fun>


In [9]:
let emptyenv: env = fun x -> Unbound;;

(* Estende l'ambiente s, aggiungendo il legame dell'identificatore x al valore v,
 * eventualmente nascondendo un legame precedente.
 *)
let bind (s: env) (x: ide) (v: evT): env = fun i -> if i = x then v else s i;;

val emptyenv : env = <fun>


val bind : env -> ide -> evT -> env = <fun>


In [10]:
let incrChannel n = if n = 255 then 255 else n + 1;;

let decrChannel n = if n = 0 then 0 else n - 1;;

exception Err of string

let rec eval (e: expr) (s: env): evT =
  match e with
    | Ide(x) -> s x
    | P(p) -> if checkPixel p then Pixel(p) else raise (Err "errore: non è un pixel")
    | Lighten(e) ->
      let res = eval e s in
        if typecheck TPixel res then
          match res with
            | Pixel(p) -> Pixel (incrChannel (getr p), incrChannel (getg p), incrChannel (getb p))
            | _ -> raise (Err "il typecheck è buggato")
        else
          raise (Err "errore: non è un pixel")
    | Darken(e) ->
      let res = eval e s in
        if typecheck TPixel res then
          match res with
            | Pixel(p) -> Pixel (decrChannel (getr p), decrChannel (getg p), decrChannel (getb p))
            | _ -> raise (Err "il typecheck è buggato")
        else
          raise (Err "errore: non è un pixel")
    | Let(x, e1, e2) -> eval e2 (bind s x (eval e1 s))
    (* Definizione di funzione *)
    | Fun(x, e) -> Closure(x, e, s)
    (* Applicazione di funzione *)
    | Apply(e1, e2) ->
      let f = eval e1 s in
        if typecheck TClosure f then
          match f with
            | Closure(formalArg, body, closureEnv) ->
              let actualArg = eval e2 s in
                let snew = bind closureEnv formalArg actualArg in
                  eval body snew
            | _ -> raise (Err "il typecheck è buggato")
        else
          raise (Err "errore: non è una funzione");;

val incrChannel : int -> int = <fun>


val decrChannel : int -> int = <fun>


exception Err of string


val eval : expr -> env -> evT = <fun>


In [11]:
let myexpr = Fun ("x", Lighten (Ide "x"));;
(*
let f = fun x -> lighten x in
  f (Pixel <0, 0, 0>)
*)
eval (Let("f", myexpr, Apply(Ide "f", P (0, 0, 0)))) emptyenv;;

(*
(fun x -> lighten x) (Pixel <10, 20, 30>)
*)
eval (Apply(myexpr, P (10, 20, 30))) emptyenv;;

val myexpr : expr = Fun ("x", Lighten (Ide "x"))


- : evT = Pixel (1, 1, 1)


- : evT = Pixel (11, 21, 31)
